In [ ]:
# from datasets import load_dataset, Dataset, DatasetDict, load_from_disk
# from bigdata_a3_utils import download_all_amazon_reviews, load_compressed_dataset
import polars as pl

In [ ]:
# auto_review: Dataset = load_dataset(
#     "McAuley-Lab/Amazon-Reviews-2023",
#     "raw_review_Automotive",
#     split="full",
#     trust_remote_code=True,
# )

# auto_meta: Dataset = load_dataset(
#     "McAuley-Lab/Amazon-Reviews-2023",
#     "raw_meta_Automotive",
#     split="full",
#     trust_remote_code=True,
# )

In [ ]:
# auto_review.to_parquet("amazon_review_auto.parquet")
# auto_meta.to_parquet("amazon_meta_auto.parquet")

In [ ]:
def extract_brand() -> pl.Expr:
    store_clean: pl.Then = pl.when(pl.col("store").str.strip_chars() != "").then(pl.col("store"))
    details_brand: pl.Expr = pl.col("details").str.extract(r"Brand[:\s]*([^\s,;]+)", 1)
    return pl.coalesce([store_clean, details_brand, pl.lit("Unknown")]).alias("brand")

In [ ]:
lf_review: pl.LazyFrame = pl.scan_parquet("data/raw/review/Gift_Cards.parquet")
lf_meta: pl.LazyFrame = pl.scan_parquet("data/raw/meta/Gift_Cards.parquet")

In [ ]:
lf: pl.LazyFrame = lf_review.join(lf_meta, on="parent_asin", how="left")

lf: pl.LazyFrame = lf.filter(pl.col("rating").is_in([1, 2, 3, 4, 5]))
lf = lf.filter(pl.col("text").str.strip_chars().str.len_chars() > 0)


lf = lf.with_columns([
    pl.col("text").str.count_matches(r"\b\w+\b").alias("review_length"),
    (pl.col("timestamp").cast(pl.Datetime("ms")).dt.year()).alias("year")
])

In [ ]:
lf = lf.unique(subset=["user_id", "text", "asin"], keep="first")
# lf = lf.with_columns([extract_brand()])

In [ ]:
# lf.sink_parquet("oops.parquet")
# print(lf.explain(streaming=True))